In [ ]:
#Need to run when data loading for the first time
!pip install opensearch-py

In [1]:
index =  "adidas_demo_test_0406_5"
language = "chinese"
endpoint_name = "bge-m3-2024-03-31-02-43-25-634-endpoint"
image_endpoint_name = 'image-embedding-clip-vit-base-patch32-2024-04-06-14-59-49-269'

In [ ]:
import json
from opensearch_search import add_products
from embeddings import get_embedding_sagemaker,get_image_embedding_sagemaker
file = 'aws_product_description.json'

product_info_list = []
product_embedding_list = []
image_embedding_list = []
metadatas = []
i = 0

with open(file) as f:
    items = json.load(f)
    for item in items:
        product_info = ''
        description_info = ''
        metadata = {}
        media_url = item['media_url'] if 'media_url' in item.keys() and len(item['media_url']) > 0 else ''
        product_code = item['product_code'] if 'product_code' in item.keys() and len(item['product_code']) > 0 else ''
        product_name = item['product_name'] if 'product_name' in item.keys() and len(item['product_name']) > 0 else ''
        reverse = item['reverse'] if 'reverse' in item.keys() and len(item['reverse']) > 0 else ''
        
        formatted_description_info = item['formatted_description_info'] if 'formatted_description_info' in item.keys() and len(item['formatted_description_info']) > 0 else ''
        if 'bullets' in formatted_description_info.keys():
            bullets = formatted_description_info['bullets']
            if len(bullets) > 0:
                bullets_info = ','.join([bullet.replace('•   ','') for bullet in bullets])
                product_info += bullets_info
                description_info += bullets_info
                
        if 'contents' in formatted_description_info.keys():
            contents = formatted_description_info['contents']
            if len(contents) > 0:
                for content in contents:
                    if 'text' in content.keys() and len(content['text']) > 0:
                        description_info += (',' + content['text'].strip())
        
        if 'title' in formatted_description_info.keys():
            title = formatted_description_info['title'].strip()
            if len(title) > 0:
                description_info += (','+title)
        
        if len(product_name) > 0:
            product_info += (','+product_name)
        
        if len(product_info) > 0: 
            product_info_list.append(product_info)
            
            product_embedding = get_embedding_sagemaker(endpoint_name, product_info, language)
            product_embedding_list.append(product_embedding)
            metadata['media_url'] = media_url
            metadata['product_code'] = product_code
            metadata['product_name'] = product_name
            metadata['reverse'] = reverse
            metadata['description_info'] = description_info
            metadatas.append(metadata)
            image_embedding = get_image_embedding_sagemaker(image_endpoint_name,media_url)
            image_embedding_list.append(image_embedding)
        
        i += 1
        print('i:',i)
        if i % 100 == 0:
            add_products(index,product_info_list,product_embedding_list,metadatas,image_embedding_list)
            product_info_list = []
            product_embedding_list = []
            image_embedding_list = []
            metadatas = []
            print('finish add products to opensearch,index is:' + index)
            
        
print('product number:',len(product_info_list))

add_products(index,product_info_list,product_embedding_list,image_embedding_list,metadatas)

print('finish add products to opensearch,index is:' + index)